In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [1]:
import torch

cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np

totalbars=32 #total bars as input
NUM_PITCHES=4 # all possible notes to play


NOTESPERBAR=16
TOTAL_NOTES=NOTESPERBAR*totalbars

num_features=NUM_PITCHES #size of input feature vector

teacherforcing=False #not used but it will be needed


In [2]:



#Generating dummy data
a= np.random.randint(NUM_PITCHES, size=TOTAL_NOTES)
print(a)
data = np.zeros((TOTAL_NOTES, NUM_PITCHES))
data[np.arange(TOTAL_NOTES), a] = 1 #generating dummy data


#adding 1 dimension, 1x32x4
data = data[np.newaxis, :, :]

data.shape

[1 0 3 2 0 0 3 1 3 0 3 0 2 0 2 1 0 3 1 2 0 2 3 0 2 3 2 0 1 3 2 0 0 0 0 2 0
 1 3 2 1 2 2 3 2 1 1 2 3 0 0 0 0 2 0 1 2 1 2 0 0 1 0 3 0 2 0 1 0 0 1 3 0 0
 3 0 3 0 1 2 3 2 1 2 2 1 0 1 3 2 3 1 3 3 2 0 1 0 2 2 0 3 2 2 3 3 0 0 1 1 0
 3 3 2 3 3 2 0 2 0 0 2 2 3 1 1 2 1 3 1 1 2 2 0 3 2 2 2 3 3 2 0 1 2 2 0 0 3
 0 3 3 1 0 2 0 0 1 3 3 3 2 2 1 2 1 3 2 1 0 3 1 3 2 0 3 3 2 1 3 1 2 1 2 2 1
 1 2 3 3 2 0 0 0 3 1 1 0 2 0 1 2 0 3 2 2 2 1 3 0 0 0 1 3 1 2 0 0 3 3 0 2 3
 1 3 2 3 2 3 1 0 1 2 1 0 3 1 1 1 1 3 2 1 3 0 2 0 1 1 1 1 2 2 0 3 0 2 0 3 1
 2 3 3 1 1 2 1 0 0 2 1 3 2 0 0 0 3 2 3 3 0 3 0 3 1 2 1 0 2 0 0 0 2 0 3 2 3
 0 1 3 1 1 3 3 1 3 2 1 2 3 2 2 1 3 0 1 2 1 3 3 1 3 3 2 3 1 1 0 1 1 1 2 0 3
 0 2 3 2 3 1 0 1 3 3 1 2 3 2 1 3 1 2 3 0 2 1 2 2 1 1 2 3 0 1 3 1 1 0 3 2 3
 2 1 1 1 3 2 2 3 1 3 0 0 0 0 0 0 1 0 1 1 2 1 3 1 1 2 3 2 0 1 3 0 1 2 0 2 3
 0 0 2 3 1 3 2 0 3 2 2 0 2 1 3 0 2 0 0 2 0 3 0 2 2 0 1 0 2 0 0 0 1 0 0 0 1
 2 2 3 0 3 2 3 1 0 1 0 3 0 1 2 0 1 0 1 2 2 0 2 2 3 3 3 3 1 1 2 1 1 1 3 2 1
 2 3 0 0 3 2 3 3 1 3 3 0 

(1, 512, 4)

In [3]:
#half of this is not yet needed but maybe it will be to visualize the latent space
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
from sklearn.decomposition import PCA
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.dark_palette("purple"))

MODEL:

In [54]:
import torch.nn as nn
from torch.nn.functional import softplus

# define size variables
input_size = NUM_PITCHES

enc_hidden_size=2048 #hidden size of encoder
conductor_hidden_size=1024 #hidden size of decoder


decoders_hidden_size=1024 #hidden size of decoder
decoders_initial_size=512 #decoder input size

n_embeddings=16

n_layers_conductor=2
n_layers_decoder=3

latent_features=512 #latent space dimension



class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_features):
        super(VariationalAutoencoder, self).__init__()
        
        self.latent_features = latent_features
       
        
        
        #data goes into bidirectional encoder
        self.encoder = torch.nn.LSTM(
                batch_first = True,
                input_size = input_size,
                hidden_size = enc_hidden_size,
                num_layers = 1,
                bidirectional = True)
        
        #encoded data goes onto connect linear layer. inputs must be*2 because LSTM is bidirectional
        #output must be 2*latentspace because it needs to be split into miu and sigma right after.
        self.encoderOut = nn.Linear(in_features=enc_hidden_size*2, out_features=latent_features*2)
        
        
        #self.decoderIn = nn.Linear(in_features=latent_features, out_features=conductor_initial_size)
        
        # The latent code must be decoded into the original image
        self.conductor = torch.nn.LSTM(
                batch_first=True,
                input_size=self.latent_features,
                hidden_size=conductor_hidden_size,
                num_layers=2,
                bidirectional = False)
                
        #Linear layer after leaving condunctor
        self.conductorout = nn.Linear(in_features=TOTAL_NOTES, out_features=2*n_embeddings)    
        
        
        # The latent code must be decoded into the original structure
        self.decoders = [torch.nn.LSTM(
                batch_first=True,
                input_size=decoders_initial_size,
                hidden_size=decoders_hidden_size,
                num_layers=2,
                bidirectional = False)] * n_embeddings

        
    #used to initialize the hidden layer of the encoder to zero before every batch
    def init_hidden(self, batch_size):
        #must be 2 x batch x hidden_size because its a bi-directional LSTM
        init = torch.zeros(2, batch_size, enc_hidden_size, device=device)
        c0 = torch.zeros(2, batch_size, enc_hidden_size, device=device)
    
        #2 because has 2 layers
        init_conductor = torch.zeros(n_layers_conductor, batch_size, conductor_hidden_size, device=device)
        c_condunctor = torch.zeros(n_layers_conductor, batch_size, conductor_hidden_size, device=device)
    
        #2 because has 2 layers
        init_decoders = torch.zeros(n_layers_decoder, batch_size, decoders_hidden_size, device=device)
        c_decoders= torch.zeros(n_layers_decoder, batch_size, decoders_hidden_size, device=device)
        
        
        return init,c0,init_conductor,c_condunctor,init_decoders,c_decoders


    def forward(self, x): 
        outputs = {}
        
        batch_size = x.size(0)
        
        print("Input size",x.size())
        
        #creates hidden layer values
        h0,c0,hconductor,cconductor,hdecoders,cdecoders = self.init_hidden(batch_size)
        
        #resets encoder at the beginning of every batch and gives it x
        x, hidden = self.encoder(x, ( h0,c0))
          
        print("after enconder",x.shape)
        
        #goes from 4096 to 1024
        x = self.encoderOut(x)
        
        print("after encoder linear",x.shape)
        
        
        
        # Split encoder outputs into a mean and variance vector 
        mu, log_var = torch.chunk(x, 2, dim=-1)
                
        # Make sure that the log variance is positive
        log_var = softplus(log_var)
               
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, 1, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        #setting sigma
        sigma = torch.exp(log_var*2)
        
       
        #generate z - latent space
        z = mu + epsilon * sigma        
        
        print("z space generated",z.shape)
        
        
               
        # Run through decoder
        x, hidden = self.conductor(z, ( hconductor,cconductor))
        
        print("conductor",x.shape)
        
        x = x.permute(0, 2, 1)
        
        #linear layer that decreases dimension
        x = self.conductorout(x)
        
        x = x.permute(0, 2, 1)
        
        
        print("after linear",x.shape)
        
        # Split encoder outputs into a mean and variance vector 
        mu_embeddings, log_var_embeddings = torch.chunk(x, 2, dim=-2)
        
        
        
        
        # Make sure that the log variance is positive
        log_var_embeddings = softplus(log_var_embeddings)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            
            epsilon_embeddings = torch.randn(mu.size(0), 1, conductor_hidden_size)
            
            if cuda:
                epsilon_embeddings = epsilon_embeddings.cuda()
        
        
        sigma_embeddings = torch.exp(log_var_embeddings/2)
        

        
        #generates embeddings
        embeddings = mu_embeddings + epsilon_embeddings * sigma_embeddings     
        
        outputs["embeddings"] = embeddings
        
        print("generated embeddings",embeddings.shape)
        
                  
        
        if torch.cuda.is_available():
            embeddings=embeddings.cuda();
            print("TYPE EMBEDDS",type(embeddings))
        
        
        for c in range(embeddings.shape[1]):
            
           
                

            
            if c == 0:
                inputs = torch.cat((embeddings[:,c,:],torch.zeros(batch_size,32,4)), 1)
            else:
                inputs = torch.cat((embeddings[:,c,:],twobar), 1)
            
            
            
            
            twobar, hidden = self.decoders[c](inputs, (hdecoders,cdecoders))
            

        
      
        
        # The original digits are on the scale [0, 1]
        x = torch.sigmoid(x)
        
        # Mean over samples
        x_hat = torch.mean(x, dim=1)
        
        outputs["x_hat"] = x_hat
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        return outputs




net = VariationalAutoencoder(latent_features)

# Transfer model to GPU if available
if cuda:
    net = net.cuda()

print(net)

VariationalAutoencoder(
  (encoder): LSTM(4, 2048, batch_first=True, bidirectional=True)
  (encoderOut): Linear(in_features=4096, out_features=1024, bias=True)
  (conductor): LSTM(512, 1024, num_layers=2, batch_first=True)
  (conductorout): Linear(in_features=512, out_features=32, bias=True)
)


Optimizer

In [55]:
#directly taken from notebook, probably some adaptation might be needed

from torch.nn.functional import binary_cross_entropy
from torch import optim



def ELBO_loss(y, t, mu, log_var):
    # Reconstruction error, log[p(x|z)]
    # Sum over features
    likelihood = -binary_cross_entropy(y, t, reduction="none")
    likelihood = likelihood.view(likelihood.size(0), -1).sum(1)

    # Regularization error: 
    # Kulback-Leibler divergence between approximate posterior, q(z|x)
    # and prior p(z) = N(z | mu, sigma*I).
    
    # In the case of the KL-divergence between diagonal covariance Gaussian and 
    # a standard Gaussian, an analytic solution exists. Using this excerts a lower
    # variance estimator of KL(q||p)
    kl = -0.5 * torch.sum(1 + log_var - mu**2 - torch.exp(log_var), dim=1)

    # Combining the two terms in the evidence lower bound objective (ELBO) 
    # mean over batch
    ELBO = torch.mean(likelihood) - torch.mean(kl)
    
    # notice minus sign as we want to maximise ELBO
    return -ELBO, kl.sum()



# define our optimizer
# The Adam optimizer works really well with VAEs.
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = ELBO_loss

Testing if forward pass works

In [56]:
from torch.autograd import Variable

#setting dummy data
x = data



#setting input ans tensor variable
x = Variable(torch.Tensor(x))

if cuda:
    x = x.cuda()
    
    

#running forward pass
outputs = net(x)


#AFTER THIS NOTHING IS DONE YET ------
x_hat = outputs["x_hat"]
mu, log_var = outputs["mu"], outputs["log_var"]
z = outputs["z"]

loss, kl = loss_function(x_hat, x, mu, log_var)

print(x.shape)
print(x_hat.shape)
print(z.shape)
print(loss)



                
                
        
print(kl)

Input size torch.Size([1, 512, 4])
after enconder torch.Size([1, 512, 4096])
after encoder linear torch.Size([1, 512, 1024])
z space generated torch.Size([1, 512, 512])
conductor torch.Size([1, 512, 1024])
after linear torch.Size([1, 32, 1024])
generated embeddings torch.Size([1, 16, 1024])
TYPE EMBEDDS <class 'torch.Tensor'>


RuntimeError: Expected a Tensor of type torch.cuda.FloatTensor but found a type torch.FloatTensor for sequence element 1 in sequence argument at position #1 'tensors'

TRAINING

In [ ]:
import os

num_epochs = 100
tmp_img = "tmp_vae_out.png"
show_sampling_points = False

train_loss, valid_loss = [], []
train_kl, valid_kl = [], []

device = torch.device("cuda:0" if cuda else "cpu")
print("Using device:", device)

for epoch in range(num_epochs):
    batch_loss, batch_kl = [], []
    net.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for x, y in train_loader:
        x = Variable(x)
        
        # This is an alternative way of putting
        # a tensor on the GPU
        x = x.to(device)
        
        outputs = net(x)
        x_hat = outputs['x_hat']
        mu, log_var = outputs['mu'], outputs['log_var']

        elbo, kl = loss_function(x_hat, x, mu, log_var)
        
        optimizer.zero_grad()
        elbo.backward()
        optimizer.step()
        
        batch_loss.append(elbo.item())
        batch_kl.append(kl.item())

    train_loss.append(np.mean(batch_loss))
    train_kl.append(np.mean(batch_kl))

    # Evaluate, do not propagate gradients
    with torch.no_grad():
        net.eval()
        
        # Just load a single batch from the test loader
        x, y = next(iter(test_loader))
        x = Variable(x)
        
        x = x.to(device)
        
        outputs = net(x)
        x_hat = outputs['x_hat']
        mu, log_var = outputs['mu'], outputs['log_var']
        z = outputs["z"]

        elbo, kl = loss_function(x_hat, x, mu, log_var)
        
        # We save the latent variable and reconstruction for later use
        # we will need them on the CPU to plot
        x = x.to("cpu")
        x_hat = x_hat.to("cpu")
        z = z.detach().to("cpu").numpy()

        valid_loss.append(elbo.item())
        valid_kl.append(kl.item())